In [1]:
from main import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import logging
from datetime import timedelta, datetime

logging.getLogger('pdfminer').setLevel(logging.ERROR)

In [2]:
receipts_path = "data/receipts"
receipts = [os.path.join(receipts_path, f) for f in os.listdir(receipts_path) if f.endswith('.pdf')]

all_receipts = []
all_products = []

for receipt in receipts:
    r, p = parse_receipt(receipt)
    all_receipts.append(r)
    all_products.append(p)

raw_receipts_df = pd.concat(all_receipts, ignore_index=True)
raw_receipts_df.to_csv("data/receipts.csv", index=False)

raw_products_df = pd.concat(all_products, ignore_index=True)
raw_products_df.to_csv("data/scanned_items.csv", index=False)

In [3]:
scanned_items_df = pd.read_csv("data/scanned_items.csv")
print("Scanned items:", len(scanned_items_df))

articles_df = scanned_items_df[['article_number', 'article_name']].drop_duplicates(subset=['article_number'])
print("Articles:", len(articles_df))

articles_df['category'] = None

articles_df.to_csv("data/articles.csv", index=False)

Scanned items: 3538
Articles: 1161


In [4]:
df = pd.read_csv('data/articles.csv')

dairy_kw = ['mjölk', 'ost', 'yoghurt', 'kvarg', 'smör', 'grädde', 'fraiche']
meat_kw = ['biff', 'fläsk', 'kyckling', 'skinka', 'korv', 'salsiccia', 'bacon', 'kött', 'fransyska', 'färs']
fish_kw = ['fisk', 'lax', 'tonfisk', 'torsk', 'makrill']
produce_kw = ['banan', 'äpple', 'apelsin', 'tomat', 'potatis', 'citron', 'lime', 'melon', 'mango', 'kiwi', 'avocado', 'paprika', 'gurka', 'lök', 'onion', 'vitlök', 'frukt']

def categorize(name):
    lname = name.lower()
    for kw in dairy_kw:
        if kw in lname:
            return 'Dairy'
    for kw in meat_kw:
        if kw in lname:
            return 'Meat'
    for kw in fish_kw:
        if kw in lname:
            return 'Fish'
    for kw in produce_kw:
        if kw in lname:
            return 'Produce'
    return

df['category'] = df['article_name'].apply(categorize)

df.to_csv("data/pre_categorized_articles.csv", index=False)